In [1]:
using DataFrames, CSV, Query #Data handling
using Convex, GLPKMathProgInterface # Optimization tools

fn = "MOWOG1entries.csv"
c1_list=["AS", "BS", "CS", "DS", "ES", "FS", "GS", "HS","SS","SSR"] # Combined 1 classes
_fill_empty_classes = true
_run_groups = 2 #Number of run groups for the event
_max_to_bump = 4 #Maximum number of entrants in the class that will still be bumped to combined
_max_driver_diff = 4 #Maximum difference in number of drivers per run group
_max_novice_diff = 5 #Maximum difference in number of novice drivers per run group
_show_empty_classes = true

#Distribute an integer over N integer parts
function distribute_int(a::T,n::T) where {T<:Integer}
    (num,den) = divrem(a,n)
    [ifelse(i<=den,num+1,num) for i=1:n]
end
@assert distribute_int(10,3)==[4,3,3]

function index_class(a::T) where {T<:DataFrameRow}
    if ismissing(a[:Index])
        if a[:Class]=="N"
            'N'*a[:LastName][1] |> String
        else
            a[:Class] |> String
        end
    else
        a[:Index] |> String
    end
end

function rungroup(a::Convex.AbstractExprOrValue,df::DataFrame, n::Integer)
    @assert n<=size(a,2)
    A = evaluate(a[:,n]) .≈ 1.0
    novice_ind = A .& df[:Novice]
    n1 = findfirst(novice_ind) |> i->df[i,:Class][2]
    n2 = findlast(novice_ind) |> i->df[i,:Class][2]
    ind = A .& .!(df[:Novice]) |> find
    y=df[ind,:Class]
    push!(y,"Novice $n1-$n2")
end

rungroup (generic function with 1 method)

In [2]:
if _show_empty_classes
    df_classes=DataFrame(Class=copy(c1_list),ClassGroup="Street")
    append!(df_classes,DataFrame(Class=('A':'F').*'M',ClassGroup="Modified"))
    append!(df_classes,DataFrame(Class="KM",ClassGroup="Modified"))
    append!(df_classes,DataFrame(Class=('C':'F').*'P',ClassGroup="Prepared"))
    append!(df_classes,DataFrame(Class="XP",ClassGroup="Prepared"))
    append!(df_classes,DataFrame(Class=('A':'F').*"SP",ClassGroup="Street Prepared"))
    append!(df_classes,DataFrame(Class=["SSM","SM","SMF"],ClassGroup="Street Modified"))
    append!(df_classes,DataFrame(Class=["STS","STH","STX","STR","STU","STP"],ClassGroup="Street Touring"))
    append!(df_classes,DataFrame(Class="CAM-".*['S','T','C'],ClassGroup="CAM"))
    # Give novices a class based on last name initial
    df_classes=vcat(df_classes,DataFrame(Class='N'.*('A':'Z'),ClassGroup=missing));
else
    df_classes=DataFrame(Class='N'.*('A':'Z'),ClassGroup=missing);
end

,Class,ClassGroup
1,AS,Street
2,BS,Street
3,CS,Street
4,DS,Street
5,ES,Street
6,FS,Street
7,GS,Street
8,HS,Street
9,SS,Street
10,SSR,Street


In [3]:
df=CSV.read(fn); #Read the CSV to a DataFrame
rename!(df, Symbol("Modifier/PAX") => :Index)
# rename!(df, Symbol("First Name")=> :FirstName)
rename!(df, Symbol("Last Name")=> :LastName)
delete!(df, [Symbol("Segment Name"),:Group])
df[:IndexClass]=map(x->index_class(x),eachrow(df))
head(df)

,LastName,Class,Year,Make,Model,Index,IndexClass
1,Ag,GS,2013,Ford,Focus ST,Z,Z
2,An,HS,2015,Ford,Fiesta ST,Z,Z
3,Au,ES,2004,Toyota,MR2,P,P
4,Ba,BS,1992,Chevrolet,Corvette,missing,BS
5,Ba,STS,1988,Honda,CRX,P,P
6,Ba,N,1996,Lexus,sc400,missing,NB


Randomly assign 23 entrants an exempt work position for testing purposes
with a preset random seed

In [4]:
n_drivers=nrow(df);
exempt_drivers=fill(false,n_drivers);
srand(562161);
exempt_drivers[randperm(n_drivers)[1:23]]=true;
df[:Exempt]=exempt_drivers;
head(df)

,LastName,Class,Year,Make,Model,Index,IndexClass,Exempt
1,Ag,GS,2013,Ford,Focus ST,Z,Z,false
2,An,HS,2015,Ford,Fiesta ST,Z,Z,false
3,Au,ES,2004,Toyota,MR2,P,P,true
4,Ba,BS,1992,Chevrolet,Corvette,missing,BS,false
5,Ba,STS,1988,Honda,CRX,P,P,false
6,Ba,N,1996,Lexus,sc400,missing,NB,false


In [5]:
#Count up the drivers per class
df=@from i in df begin
    @group i by i.IndexClass into g
    @orderby ascending(g.key)
    @select {Class=g.key, Drivers=length(g), Exempt=sum(g..Exempt)}
    @collect DataFrame
end
head(df)

,Class,Drivers,Exempt
1,ASP,1,0
2,BS,7,1
3,CAM-S,3,2
4,CAM-T,3,1
5,CS,3,1
6,DM,2,0


In [6]:
df_classes[:Drivers]=0; df_classes[:Exempt]=0;
df_classes[[:Class,:Drivers,:Exempt]]
sum(df[:Drivers])
df=vcat(df, df_classes[[:Class,:Drivers,:Exempt]])
df=join(df,df_classes[[:Class,:ClassGroup]],on=:Class,kind=:left)

,Class,Drivers,Exempt,ClassGroup
1,ASP,1,0,Street Prepared
2,BS,7,1,Street
3,CAM-S,3,2,CAM
4,CAM-T,3,1,CAM
5,CS,3,1,Street
6,DM,2,0,Modified
7,DS,4,0,Street
8,EM,2,1,Modified
9,ES,5,1,Street
10,FS,1,0,Street


In [7]:
(@from i in df begin
    @where i.Drivers==0
    @select i
    @collect DataFrame
    end) |> head

,Class,Drivers,Exempt,ClassGroup
1,AS,0,0,Street
2,BS,0,0,Street
3,CS,0,0,Street
4,DS,0,0,Street
5,ES,0,0,Street
6,FS,0,0,Street


In [8]:
# Sum drivers per class, to keep the empty classes active
df=@from i in df begin
    @group i by i.Class into g
    @select {Class=g.key, Drivers=sum(g..Drivers), Exempt=sum(g..Exempt), Novice=g.key[1]=='N',
        ClassGroup=first(g..ClassGroup)}
    @collect DataFrame
end
head(df)

,Class,Drivers,Exempt,Novice,ClassGroup
1,ASP,1,0,false,Street Prepared
2,BS,7,1,false,Street
3,CAM-S,3,2,false,CAM
4,CAM-T,3,1,false,CAM
5,CS,3,1,false,Street
6,DM,2,0,false,Modified


In [9]:
df=@from i in df begin
    @orderby ascending(i.Novice),ascending(i.ClassGroup),ascending(i.Class)
    @select i
    @collect DataFrame
end;

In [10]:
# let d=df[find(x->x[:Class]=="N",eachrow(df)),:]
#     DataFrame(Class=(@. "N"*string(1:_run_groups)),
#         Drivers=distribute_int(d[:Drivers][1],_run_groups),
#         Exempt=distribute_int(d[:Exempt][1],_run_groups)) |> x -> append!(df,x)
# end

df=@from i in df begin
    @where i.Class != "N"
    @select i
    @collect DataFrame
end
df[:Workers]=df[:Drivers].-df[:Exempt]
head(df)

,Class,Drivers,Exempt,Novice,ClassGroup,Workers
1,CAM-C,0,0,false,CAM,0
2,CAM-S,3,2,false,CAM,1
3,CAM-T,3,1,false,CAM,2
4,AM,0,0,false,Modified,0
5,BM,0,0,false,Modified,0
6,CM,0,0,false,Modified,0


In [11]:
# Create our variables
N = nrow(df)
x = Variable((N,_run_groups), :Bin) #Class allocation variable

Variable of
size: (72, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [12]:
#Each class must be in exactly 1 run group
constr=sum(x,2).==1;

This next set constrains the Novice class split so that Run Group #1 starts a A and continues to a "L1",
and Group #2 resumes form "L1+1" to "L2", and Group #3 etc... resumes from "L2+1" to Z.

An illustration of the constraints is shown below, for an example of 4 letters, and 3 run groups

In [13]:
let N=4, _run_groups=3
    ["sum(x[$i,1:$n])>=x[$(i+1),$n]" for n=1:_run_groups-1 for i=1:N-1] .|> println
end;

sum(x[1,1:1])>=x[2,1]
sum(x[2,1:1])>=x[3,1]
sum(x[3,1:1])>=x[4,1]
sum(x[1,1:2])>=x[2,2]
sum(x[2,1:2])>=x[3,2]
sum(x[3,1:2])>=x[4,2]


In [14]:
sub_x = x[df[:Novice] |> find,:]  # sub_array of only the novice classes
constr+=(x->[sum(x[i,1:n])>=x[(i+1),n] for n=1:_run_groups-1 for i=1:size(x,1)-1])(sub_x);

In [15]:
#keep Combined classes together if necessary
constr+=let ind=[any(d[:Class].==c1_list) && (d[:Drivers] <= _max_to_bump) && !d[:Novice] for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end;
constr+=let ind=[!any(d[:Class].==c1_list) && (d[:Drivers] <= _max_to_bump) && !d[:Novice] for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end;

Combining AS,CS,DS,FS,GS,HS,SS,SSR due to <= 4 drivers
Combining CAM-C,CAM-S,CAM-T,AM,BM,CM,DM,EM,FM,KM,CP,DP,EP,FP,XP,SM,SSM,ASP,BSP,CSP,DSP,ESP,FSP,STH,STP,STR,STS,STU,STX,SSC,SSP,V,X due to <= 4 drivers


In [16]:
#Split Pro & Z
constr+=let ind=[any(d[:Class].==["P","Z"]) for d in eachrow(df)] |> find
    sum(x[ind,:],1).<=1
end;

In [17]:
#Expressions that can be used in the optimizer
rungroup_workers=sum(x.*df[:Workers],1) |> vec #Workers available per run group
rungroup_drivers=sum(x.*df[:Drivers],1) |> vec #Drivers in each run group
rungroup_novice= sum(Vector(df[:Drivers].*df[:Novice]).*x,1) |> vec

AbstractExpr with
head: reshape
size: (2, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [18]:
constr+=maximum(rungroup_drivers)-minimum(rungroup_drivers)<=_max_driver_diff;
constr+=maximum(rungroup_novice)-minimum(rungroup_novice)<=_max_novice_diff;

In [19]:
# Define the problem's optimization, under required constraints
p=maximize(minimum(rungroup_workers),constr);  #Maximize, the Minimum # of workers in a run group

In [20]:
solve!(p, GLPKSolverMIP())

In [21]:
#What is the status of the solutioin
p.status

:Optimal

In [22]:
let f = x-> Integer.(evaluate(x))
    for (i,(drivers,novice,workers)) in enumerate(zip(f.((rungroup_drivers,rungroup_novice,rungroup_workers))...))
        println("Run group #$i, $drivers drivers, $workers workers, $novice novice drivers")
    end
end

Run group #1, 68 drivers, 58 workers, 17 novice drivers
Run group #2, 71 drivers, 58 workers, 12 novice drivers


In [23]:
println("Run Group #1:"); println.(rungroup(x,df,1));

Run Group #1:
AS
BS
CS
DS
ES
FS
GS
HS
SS
SSR
SMF
P
Novice A-L


In [24]:
println("Run Group #2:"); println.(rungroup(x,df,2));

Run Group #2:
CAM-C
CAM-S
CAM-T
AM
BM
CM
DM
EM
FM
KM
CP
DP
EP
FP
XP
SM
SSM
ASP
BSP
CSP
DSP
ESP
FSP
STH
STP
STR
STS
STU
STX
SSC
SSP
V
X
Z
Novice M-Z


In [25]:
_run_groups >= 3 && (println("Run Group #3:"); println.(rungroup(x,df,3)));